In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

import xgboost as xgb
#Training will be done by RandomForest Algorithm
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler #StandardScaler function of sklearn is used to scale down values of dataframe between 0 and 1
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
display(train)
display(test)

In [ ]:
sns.pairplot(data=train)

Based on the Pairplot above, following are my 1st cut inference

1) Targets are distributed along the entire range of humidity & Temperature. They can help to cluster the targets

2) Sensor 1, 2, 4 and 5 seem to have linear relation to the targets.

3) Sensor 3 has a non linear relationships with all three targets.

Strategy : 
>> Split the data and target columns seperately.

>> Scale the data 

>> Model the data using Random Forest, Linear Regressor and Support Vector Regressor

>> Test it on the another other file, and find how well your model works.

Start with the Mutual infomation for Target_Corbon_monoxide

In [ ]:
#Remove the dates from the set, since their influence is not high
train = train.drop('date_time',axis=1)

In [ ]:
#popping out the targets from the train dataset
y_CO = train.pop('target_carbon_monoxide')
y_BE = train.pop('target_benzene')
y_NO = train.pop('target_nitrogen_oxides')

In [ ]:
#Checking dataset before Scaling the training dataset.This will be the input to the model
display(train)

In [ ]:
#Scaling using MinMax Scaler
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train)

In [ ]:
train[0]

In [ ]:
#Training data after scaling is an numpy array datatype
print(train)
print(type(train))

With the raw data for the model building ready, it  has to be split into 3 groups.
1) Training Dataset
2) Cross Validation Dataset
3) Testing Dataset

Why split this way? 
In this competition the goal is to reach the highest RMSE score on the test data, that is shared with all the competition. What is not shared is the sensor output based on the test data, on which the RMSE score will be calculated.

You assume the role of the competition host, and not share some of the data and results to the models that you are training. This way, you can have a "Mock Exam" with the models before submitting to the real competition.  

In [ ]:
X_train , X_cv , y_train_co , y_cv_co  = train_test_split(train ,y_CO, test_size = 0.2)
X_train , X_test , y_train_co , y_test_co  = train_test_split(X_train ,y_train_co, test_size = 0.2)

In [ ]:
#Working only on the remaining NO and Benezene targets
y_train_no , y_cv_no  = train_test_split(y_NO, test_size = 0.2)
y_train_no , y_test_no  = train_test_split(y_train_no, test_size = 0.2)

In [ ]:
y_train_be , y_cv_be  = train_test_split(y_BE, test_size = 0.2)
y_train_be , y_test_be  = train_test_split(y_train_be, test_size = 0.2)

In [ ]:
# Target carbon Monoxide columns has been also split same as training data.
# Ensure the lengths of all the datasets are same
print(X_train.shape)
print(X_cv.shape)
print(X_test.shape)
print(y_train_co.shape)
print(y_cv_co.shape)
print(y_test_co.shape)

Running the MI Scores for the Target Carbon Monoxides

Sensor 3 is not linear as per the above patter. The components of this sensor can render a different result. So doing PCA after the Linear Regression

In [ ]:
#Fitting the data that matters
CO_r = LinearRegression(normalize=True)
CO_r.fit(X_train,y_train_co)
y_pred_co = CO_r.predict(X_cv)
#I am getting negative values when predicting only the CO values. So making them 0
y_pred_co[y_pred_co < 0] = 0 
print("RMSE with simple Linear Regression",mean_squared_log_error(y_cv_co,y_pred_co))
BE_r = LinearRegression(normalize=True)
BE_r.fit(X_train,y_train_be)
y_pred_be = BE_r.predict(X_cv)
print("RMSE with simple Linear Regression",mean_squared_log_error(y_cv_be,y_pred_be))
NO_r = LinearRegression(normalize=True)
NO_r.fit(X_train,y_train_no)
y_pred_no = NO_r.predict(X_cv)
print("RMSE with simple Linear Regression",mean_squared_log_error(y_cv_no,y_pred_no))


In [ ]:
d = {'CO': list(y_cv_co), 'BE': list(y_cv_be), 'NO':list(y_cv_no)}
y_cv = pd.DataFrame(data=d)
y_cv.head()

In [ ]:
p = {'CO': list(rand_yco), 'BE': list(rand_ybe), 'NO':list(rand_yno)}
y_rand = pd.DataFrame(data=p)
y_rand.head()

In [ ]:
y_cv.shape

#Using RandomForest Regressor

In [ ]:
#Carbon Monoxide
CO=RandomForestRegressor(n_estimators=50,random_state=0,criterion="mse")
CO.fit(X_train,y_train_co)
rand_yco=CO.predict(X_cv)
print("mean SQ error CO:",mean_squared_log_error(rand_yco,y_cv_co))
#Benzene
BE=RandomForestRegressor(n_estimators=50,random_state=0,criterion="mse")
BE.fit(X_train,y_train_be)
rand_ybe=BE.predict(X_cv)
print("mean SQ error BE:",mean_squared_log_error(rand_ybe,y_cv_be))
#Nitrous Oxide
NO=RandomForestRegressor(n_estimators=50,random_state=0,criterion="mse")
NO.fit(X_train,y_train_no)
rand_yno=NO.predict(X_cv)
print("mean SQ error NO:",mean_squared_log_error(rand_yno,y_cv_no))

After the PCA there is not much improvement in the scores, so going ahead with existing data

In [ ]:
#Carbon Monoxide
s_CO=SVR()
s_CO.fit(X_train,y_train_co)
s_yco=s_CO.predict(X_cv)
print("mean SQ error CO:",mean_squared_log_error(s_yco,y_cv_co))
#Benzene
s_BE=SVR()
s_BE.fit(X_train,y_train_be)
s_ybe=s_BE.predict(X_cv)
print("mean SQ error BE:",mean_squared_log_error(s_ybe,y_cv_be))
#Nitrous Oxide
s_NO=SVR()
s_NO.fit(X_train,y_train_no)
s_yno=s_NO.predict(X_cv)
print("mean SQ error NO:",mean_squared_log_error(s_yno,y_cv_no))

## Based on the SVR, Random Regressor and Linear Regressor, the RMSE for SVR has come low

In [ ]:
#Predicting Test Carbon Monoxide with SVR
test_yco=s_CO.predict(X_test)
print("mean SQ error CO:",mean_squared_log_error(test_yco,y_test_co))
#Predicting Test Benzene with SVR
test_ybe=s_BE.predict(X_test)
print("mean SQ error BE:",mean_squared_log_error(test_ybe,y_test_be))
#Predicting Test Nitrous Oxide with SVR
test_yno=s_NO.predict(X_test)
print("mean SQ error NO:",mean_squared_log_error(test_yno,y_test_no))

Mean SQ error with the test data on the SVR model has yielded the lower RMSE value

In [ ]:
test.head()

In [ ]:
#Scaling the Actual test dataframe after dropping the date column
test = test.drop('date_time',axis=1)

In [ ]:
trial = pd.DataFrame()
trial['target_carbon_monoxide'] = list(test_yco)
trial['target_benzene'] = list(test_ybe)
trial['target_nitrogen_oxides'] = list(test_yno)

In [ ]:
mean_squared_log_error(trial,sim)

In [ ]:
sim = pd.DataFrame()
sim['target_carbon_monoxide'] = list(y_test_co)
sim['target_benzene'] = list(y_test_be)
sim['target_nitrogen_oxides'] = list(y_test_no)

In [ ]:
sim.head()

In [ ]:
#Scaling using MinMax Scaler
scaler = MinMaxScaler(feature_range=(0, 1))
test = scaler.fit_transform(test)

In [ ]:
#Predicting Carbon Monoxide with SVR and competition test data
test_co=s_CO.predict(test)
#Predicting Benzene with SVR and competition test data
test_be=s_BE.predict(test)
#Predicting Nitrous Oxide with SVR and competition test data
test_no=s_NO.predict(test)


In [ ]:
#Reinitialising the competition test dataframe again
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
test.shape

In [ ]:
test['target_carbon_monoxide'] =  test_co #Adding the Predicted Target_CO to test dataframe

In [ ]:
test['target_benzene'] = test_be #Adding the Predicted Target_CO to test dataframe

In [ ]:
test['target_nitrogen_oxides'] = test_no#Adding the Predicted Target_CO to test dataframe

In [ ]:
submission = test[['date_time','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]

In [ ]:
submission.shape

In [ ]:
submission.to_csv('my_submission.csv',index=False)